In [ ]:
import numpy as np
from keras.applications.densenet import DenseNet121
import keras.backend as K
import cv2
import os
from keras.callbacks import LearningRateScheduler,ReduceLROnPlateau,CSVLogger,ModelCheckpoint
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score,roc_auc_score
from sklearn.metrics import cohen_kappa_score,classification_report,auc,roc_curve
from keras.callbacks import Callback
from keras.layers import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
# from tqdm import tqdm

In [ ]:
imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std = np.array([0.229, 0.224, 0.225])

def img_standardization(x):
    
#     x=cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x=x.astype('float16')/255.
#     return x
    return ((x-imagenet_mean)/imagenet_std)

In [ ]:
import imgaug.augmenters as iaa

augmentation = iaa.Sequential([
    iaa.OneOf([ ## geometric transform
        iaa.Affine(
            scale={"x": (0.98, 1.02), "y": (0.98, 1.04)},
            translate_percent={"x": (-0.02, 0.02), "y": (-0.04, 0.04)},
            rotate=(-2, 2),
            shear=(-1, 1),
        ),
        iaa.PiecewiseAffine(scale=(0.001, 0.025)),
    ]),
    iaa.OneOf([ ## brightness or contrast
        iaa.Multiply((0.9, 1.1)),
        iaa.ContrastNormalization((0.9, 1.1)),
    ]),
    iaa.OneOf([ ## blur or sharpen
        iaa.GaussianBlur(sigma=(0.0, 0.1)),
        iaa.Sharpen(alpha=(0.0, 0.1)),
    ]),
])

In [ ]:
def batch_generator(X, batch_size = 32):
    indices = np.arange(len(X))
    print (indices)
    batch=[]
    while True:
            for i in indices:
                batch.append(i)
                if len(batch)==batch_size or i==indices[-1]:
                    x=augmentation(images=X[batch])
                    x=img_standardization(x)
                    yield x
                    batch=[]

In [ ]:
augmentation1 = iaa.Affine(
            scale={"x": (0.98, 1.02), "y": (0.98, 1.04)},
            translate_percent={"x": (-0.02, 0.02), "y": (-0.04, 0.04)},
            rotate=(-2, 2),
            shear=(-1, 1),
        )

augmentation2=iaa.PiecewiseAffine(scale=(0.001, 0.025))

augmentation3=iaa.Multiply((0.9, 1.1))
augmentation4=iaa.ContrastNormalization((0.9, 1.1))

augmentation5=iaa.GaussianBlur(sigma=(0.0, 0.1))
augmentation6=iaa.Sharpen(alpha=(0.0, 0.1))

In [ ]:
def get_probs_for_all_augs(X, batch_size = 32):
    final_preds1=[]
    final_preds2=[]
    final_preds3=[]
    final_preds4=[]
    final_preds5=[]
    final_preds6=[]
    final_preds7=[]
    
    for idx, i in enumerate(X):
        x=augmentation1(images=i)
        x=img_standardization(x)
        preds1=new_model.predict(np.expand_dims(x, axis=0))
        final_preds1.append(preds1[0][1])
        
        x=augmentation2(images=i)
        x=img_standardization(x)
        preds2=new_model.predict(np.expand_dims(x, axis=0))
        final_preds2.append(preds2[0][1])
        
        x=augmentation3(images=i)
        x=img_standardization(x)
        preds3=new_model.predict(np.expand_dims(x, axis=0))
        final_preds3.append(preds3[0][1])
        
        x=augmentation4(images=i)
        x=img_standardization(x)
        preds4=new_model.predict(np.expand_dims(x, axis=0))
        final_preds4.append(preds4[0][1])
        
        x=augmentation5(images=i)
        x=img_standardization(x)
        preds5=new_model.predict(np.expand_dims(x, axis=0))
        final_preds5.append(preds5[0][1])
        
        x=augmentation6(images=i)
        x=img_standardization(x)
        preds6=new_model.predict(np.expand_dims(x, axis=0))
        final_preds6.append(preds6[0][1])
        
        x=img_standardization(i)
        preds7=new_model.predict(np.expand_dims(x, axis=0))
        final_preds7.append(preds7[0][1])

        print (idx)
    return (final_preds1, final_preds2, final_preds3, final_preds4, final_preds5, final_preds6, final_preds7)

In [ ]:
from keras.optimizers import Adam
adam=Adam(lr=1e-4)

#Model declaration:Densenet 
dense_model = DenseNet121(weights='imagenet', include_top=False,input_shape=(224,224,3),pooling='avg')
preds = Dense(14,activation='sigmoid')(dense_model.output)
model = Model(dense_model.input,preds)
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
model.load_weights('/opt/bucketdata/Users/Rohit/15_pathology_model/NIH_more_than_decent_weights/updated_best_weights.h5')


#Model parameters for compiling

model.compile(optimizer=adam,loss='binary_crossentropy',metrics=['accuracy'])


In [ ]:
new_preds = Dense(2,activation='softmax')(model.get_layer('avg_pool').output)
new_model = Model(model.input,new_preds)
new_model.compile(optimizer=adam,loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
new_model.load_weights('./Weights/CHAI_PvsNP_newlabels_transferlearned_fromNIHPvsNP.hdf5')

In [ ]:
Y_test=[]
for i in y_test:
    if i==2:
        Y_test.append(1)
    else:
        Y_test.append(i)

# Max vote TTA/Average TTA

In [ ]:
final_probs1, final_probs2, final_probs3, final_probs4, final_probs5, final_probs6, final_probs7 = get_probs_for_all_augs(x_test)

In [ ]:
! mkdir Predictions_from_valsaved_woaug_Nov
np.save('./Predictions_from_valsaved_woaug_Nov/probs_affine.npy', np.array(final_probs1))
np.save('./Predictions_from_valsaved_woaug_Nov/probs2_piecewiseaffine.npy', np.array(final_probs2))
np.save('./Predictions_from_valsaved_woaug_Nov/probs3_multiply.npy', np.array(final_probs3))
np.save('./Predictions_from_valsaved_woaug_Nov/probs4_contrast.npy', np.array(final_probs4))
np.save('./Predictions_from_valsaved_woaug_Nov/probs5_gaussian.npy', np.array(final_probs5))
np.save('./Predictions_from_valsaved_woaug_Nov/probs6_sharpen.npy', np.array(final_probs6))
np.save('./Predictions_from_valsaved_woaug_Nov/probs7_original.npy', np.array(final_probs7))

In [ ]:
y1=np.load("./Predictions_from_valsaved_woaug_Nov/probs_affine.npy",mmap_mode='r')
y2=np.load("./Predictions_from_valsaved_woaug_Nov/probs2_piecewiseaffine.npy", mmap_mode='r')

y3=np.load("./Predictions_from_valsaved_woaug_Nov/probs3_multiply.npy",mmap_mode='r')
y4=np.load("./Predictions_from_valsaved_woaug_Nov/probs4_contrast.npy", mmap_mode='r')

y5=np.load("./Predictions_from_valsaved_woaug_Nov/probs5_gaussian.npy",mmap_mode='r')
y6=np.load("./Predictions_from_valsaved_woaug_Nov/probs6_sharpen.npy",mmap_mode='r')
y7=np.load("./Predictions_from_valsaved_woaug_Nov/probs7_original.npy",mmap_mode='r')

In [ ]:
avg_preds=(y3+y4+y5+y6+y7)/5

In [ ]:
final=np.zeros((len(y1),7))
threshold=0.0305
for idx, i in enumerate(y1):
#     if y1[idx]>=threshold:
#         final[idx][0]=1
#     if y2[idx]>=threshold:
#         final[idx][1]=1
    if y3[idx]>=threshold:
        final[idx][2]=1
    if y4[idx]>=threshold:
        final[idx][3]=1
    if y5[idx]>=threshold:
        final[idx][4]=1
    if y6[idx]>=threshold:
        final[idx][5]=1
    if y7[idx]>=threshold:
        final[idx][6]=1

In [ ]:
final_vote=np.sum(final, axis=1)

In [ ]:
class_preds=[]
for i in final_vote:
    if i>=3:
        class_preds.append(1)
    else:
        class_preds.append(0)

In [ ]:
Y_pred=class_preds
# Y_test=np.argmax(Y_test,axis=-1)
target_names = ['normal','Pathology']

print("Confusion Matrix:")
print (confusion_matrix(Y_test,Y_pred))

print("Classification report:")
print(classification_report(Y_test,Y_pred,target_names=target_names))

print("Kappa score:")
print(cohen_kappa_score(Y_test,Y_pred))